In [4]:
import pandas as pd
import numpy as np
import csv
from keras.utils import to_categorical

In [21]:
test = pd.read_csv('compdata\\kaggletest.csv')
train = pd.read_csv('compdata\\kaggletrain.csv')

labels_train = train['label']

trainingset = train.drop('label', axis=1)
trainingset = trainingset/255
trainingset = trainingset.to_numpy()
trainingset = trainingset.reshape(42000,28,28,1)

testset = test.drop('label', axis=1)
testset = testset/255
testset = testset.to_numpy()
testset = testset.reshape(42000,28,28,1)


KeyError: "['label'] not found in axis"

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping



In [31]:
convModel = Sequential([
    Conv2D(filters=64,kernel_size=(4,4), input_layer=(28,28,1), padding='SAME'),
    MaxPool2D(pool_size=(4,4)),
    Conv2D(filter=64)
])
#Scaling Training Data 



(60000, 28, 28)